In [37]:
from copy import deepcopy

class CheckersGame:
    def __init__(self):
        self.board = [
            [0, 1, 0, 1, 0, 1, 0, 1],
            [1, 0, 1, 0, 1, 0, 1, 0],
            [0, 1, 0, 1, 0, 1, 0, 1],
            [0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0],
            [2, 0, 2, 0, 2, 0, 2, 0],
            [0, 2, 0, 2, 0, 2, 0, 2],
            [2, 0, 2, 0, 2, 0, 2, 0]
        ]
        self.current_player = 1

    def is_valid_move(self, start, end, player):
        if not (0 <= start[0] < 8 and 0 <= start[1] < 8 and 0 <= end[0] < 8 and 0 <= end[1] < 8):
            return False

        if self.board[start[0]][start[1]] != player:
            return False

        if self.board[end[0]][end[1]] != 0:
            return False

        if abs(start[0] - end[0]) != abs(start[1] - end[1]):
            return False

        if player == 1:
            if end[0] < start[0]:
                return False
        elif player == 2:
            if end[0] > start[0]:
                return False

        if abs(start[0] - end[0]) == 1 and abs(start[1] - end[1]) == 1:
            return True

        dx = end[0] - start[0]
        dy = end[1] - start[1]
        jump_x = start[0] + dx // 2
        jump_y = start[1] + dy // 2

        if abs(dx) == 2 and abs(dy) == 2:
            if abs(start[0] - jump_x) != 1 or abs(start[1] - jump_y) != 1:
                return False
            if self.board[jump_x][jump_y] == 0:
                return False
            if self.board[jump_x][jump_y] == player:
                return False
            return True

        return False

    def make_move(self, start, end):
        dx = end[0] - start[0]
        dy = end[1] - start[1]
        if abs(dx) == 2 and abs(dy) == 2:
            jump_x = start[0] + dx // 2
            jump_y = start[1] + dy // 2
            self.board[jump_x][jump_y] = 0

        self.board[end[0]][end[1]] = self.board[start[0]][start[1]]
        self.board[start[0]][start[1]] = 0

        if self.board[end[0]][end[1]] == 1 and end[0] == 7:
            self.board[end[0]][end[1]] = 3
        elif self.board[end[0]][end[1]] == 2 and end[0] == 0:
            self.board[end[0]][end[1]] = 4

        self.current_player = 2 if self.current_player == 1 else 1

    def get_valid_moves(self, player):
      moves = []
      for i in range(8):
        for j in range(8):
            if self.board[i][j] == player or (player == 1 and self.board[i][j] == 3) or (player == 2 and self.board[i][j] == 4):

                if player == 1 or self.board[i][j] == 3:
                    if i < 7 and j > 0 and self.board[i + 1][j - 1] == 0:
                        moves.append([(i, j), (i + 1, j - 1)])
                    if i < 7 and j < 7 and self.board[i + 1][j + 1] == 0:
                        moves.append([(i, j), (i + 1, j + 1)])
                if player == 2 or self.board[i][j] == 4:
                    if i > 0 and j > 0 and self.board[i - 1][j - 1] == 0:
                        moves.append([(i, j), (i - 1, j - 1)])
                    if i > 0 and j < 7 and self.board[i - 1][j + 1] == 0:
                        moves.append([(i, j), (i - 1, j + 1)])

                if player == 1 or self.board[i][j] == 3:
                    if i < 7 and j > 0 and self.board[i + 1][j - 1] == 2:
                        if i < 6 and j > 1 and self.board[i + 2][j - 2] == 0:
                            moves.append([(i, j), (i + 2, j - 2)])
                    if i < 7 and j < 7 and self.board[i + 1][j + 1] == 2:
                        if i < 6 and j < 6 and self.board[i + 2][j + 2] == 0:
                            moves.append([(i, j), (i + 2, j + 2)])
                if player == 2 or self.board[i][j] == 4:
                    if i > 0 and j > 0 and self.board[i - 1][j - 1] == 1:
                        if i > 1 and j > 1 and self.board[i - 2][j - 2] == 0:
                            moves.append([(i, j), (i - 2, j - 2)])
                    if i > 0 and j < 7 and self.board[i - 1][j + 1] == 1:
                        if i > 1 and j < 6 and self.board[i - 2][j + 2] == 0:
                            moves.append([(i, j), (i - 2, j + 2)])

      return moves


    def evaluate_board(self):
        score = 0
        for i in range(8):
            for j in range(8):
                if self.board[i][j] == 1:
                    score += 1
                elif self.board[i][j] == 2:
                    score -= 1
                elif self.board[i][j] == 3:
                    score += 2
                elif self.board[i][j] == 4:
                    score -= 2
        return score

    def game_over(self):
        black_pieces = 0
        red_pieces = 0
        for i in range(8):
            for j in range(8):
                if self.board[i][j] == 1 or self.board[i][j] == 3:
                    black_pieces += 1
                elif self.board[i][j] == 2 or self.board[i][j] == 4:
                    red_pieces += 1

        if black_pieces == 0:
            return True, 2
        if red_pieces == 0:
            return True, 1

        black_moves = len(self.get_valid_moves(1))
        red_moves = len(self.get_valid_moves(2))

        if black_moves == 0:
            return True, 2
        if red_moves == 0:
            return True, 1

        return False, None

    def alpha_beta_pruning(self, depth, alpha, beta, maximizing_player):
        valid_moves = self.get_valid_moves(self.current_player)

        if depth == 0 or len(valid_moves) == 0:
            return self.evaluate_board()

        if maximizing_player:
            max_eval = float('-inf')
            for move in valid_moves:
                new_board = deepcopy(self)
                new_board.make_move(move[0], move[1])
                eval = new_board.alpha_beta_pruning(depth - 1, alpha, beta, False)
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return max_eval
        else:
            min_eval = float('inf')
            for move in valid_moves:
                new_board = deepcopy(self)
                new_board.make_move(move[0], move[1])
                eval = new_board.alpha_beta_pruning(depth - 1, alpha, beta, True)
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return min_eval

    def play(self):
        while True:
            valid_moves = self.get_valid_moves(self.current_player)
            if len(valid_moves) == 0:
                game_over, winner = self.game_over()
                if game_over:
                    if winner == 1:
                        print("Black player wins!")
                    else:
                        print("Red player wins!")
                else:
                    print("It's a draw!")
                break

            best_eval = float('-inf') if self.current_player == 1 else float('inf')
            best_move = None

            for move in valid_moves:
                new_board = deepcopy(self)
                new_board.make_move(move[0], move[1])
                eval = new_board.alpha_beta_pruning(5, float('-inf'), float('inf'), self.current_player == 1)
                if (self.current_player == 1 and eval >= best_eval) or (self.current_player == 2 and eval <= best_eval):
                    best_eval = eval
                    best_move = move

            self.make_move(best_move[0], best_move[1])
            self.print_board()

    def print_board(self):
      for i in range(8):
        for j in range(8):
            if self.board[i][j] == 0:
                print("|.", end=" ")
            elif self.board[i][j] == 1:
                print("|B", end=" ")
            elif self.board[i][j] == 2:
                print("|R", end=" ")
            elif self.board[i][j] == 3:
                print("|BK", end=" ")
            else:
                print("|RK", end=" ")
        if i == 3:
            print("     Turn: Player", self.current_player, end="")
        print()
        print("-----------------------")
      print("=============================================")



if __name__ == "__main__":
    game = CheckersGame()
    game.play()


|. |B |. |B |. |B |. |B 
-----------------------
|B |. |B |. |B |. |B |. 
-----------------------
|. |B |. |B |. |B |. |. 
-----------------------
|. |. |. |. |. |. |B |.      Turn: Player 2
-----------------------
|. |. |. |. |. |. |. |. 
-----------------------
|R |. |R |. |R |. |R |. 
-----------------------
|. |R |. |R |. |R |. |R 
-----------------------
|R |. |R |. |R |. |R |. 
-----------------------
|. |B |. |B |. |B |. |B 
-----------------------
|B |. |B |. |B |. |B |. 
-----------------------
|. |B |. |B |. |B |. |. 
-----------------------
|. |. |. |. |. |. |B |.      Turn: Player 1
-----------------------
|. |. |. |. |. |. |. |R 
-----------------------
|R |. |R |. |R |. |. |. 
-----------------------
|. |R |. |R |. |R |. |R 
-----------------------
|R |. |R |. |R |. |R |. 
-----------------------
|. |B |. |B |. |B |. |B 
-----------------------
|B |. |B |. |B |. |B |. 
-----------------------
|. |B |. |B |. |B |. |. 
-----------------------
|. |. |. |. |. |. |. |.      Tu

KeyboardInterrupt: 